In [ ]:
import pandas as pd
from functools import lru_cache

# Cadastro e login
usuarios = [{
    "nome": "admin",
    "senha": "admin",
    "tempo": 24,
    "nivel": "básico"
}]

def cadastrar_usuario():
    nome = input("Digite seu nome: ")
    senha = input("Crie uma senha: ")
    tempo = int(input("Quanto tempo (em horas) você tem disponível por dia para estudar? "))
    nivel = input("Qual seu nível de conhecimento? (básico/intermediário/avançado): ")
    usuarios.append({"nome": nome, "senha": senha, "tempo": tempo, "nivel": nivel})
    print("Cadastro realizado com sucesso!\n")

def login():
    print("Faça login para acessar o sistema:")
    print("""
    usuário default:
    nome: admin
    senha: admin
    """)
    nome = input("Nome de usuário: ")
    senha = input("Senha: ")
    for user in usuarios:
        if user["nome"] == nome and user["senha"] == senha:
            print(f"Login bem-sucedido! Bem-vindo(a), {nome}\n")
            return user
    print("Usuário ou senha incorretos.")
    return None

# Base de dados (20 cursos)
cursos = [
    {"nome": "Python", "impacto": 9, "custo": 15},
    {"nome": "IA Ética", "impacto": 7, "custo": 4},
    {"nome": "Excel Avançado", "impacto": 5, "custo": 5},
    {"nome": "Power BI", "impacto": 8, "custo": 18},
    {"nome": "Machine Learning", "impacto": 10, "custo": 30},
    {"nome": "SQL", "impacto": 6, "custo": 25},
    {"nome": "Gestão Ágil", "impacto": 5, "custo": 10},
    {"nome": "Design Thinking", "impacto": 7, "custo": 5},
    {"nome": "Comunicação Profissional", "impacto": 6, "custo": 2},
    {"nome": "Inglês Técnico", "impacto": 8, "custo": 4},
    {"nome": "Finanças para TI", "impacto": 5, "custo": 8},
    {"nome": "Docker Básico", "impacto": 7, "custo": 3},
    {"nome": "Kubernetes", "impacto": 9, "custo": 5},
    {"nome": "Java Avançado", "impacto": 8, "custo": 20},
    {"nome": "Git e GitHub", "impacto": 6, "custo": 5},
    {"nome": "Redes de Computadores", "impacto": 7, "custo": 10},
    {"nome": "Segurança da Informação", "impacto": 8, "custo": 30},
    {"nome": "Liderança Técnica", "impacto": 7, "custo": 3},
    {"nome": "APIs com FastAPI", "impacto": 9, "custo": 4},
    {"nome": "Data Science", "impacto": 10, "custo": 5}
]

df_cursos = pd.DataFrame(cursos)


# Estrutura para ordenar os dados (merge sort)

def merge_sort(dataframe, coluna):
    # converte DataFrame em lista de dicts para evitar incompatibilidades
    lista = dataframe.to_dict('records')
    if len(lista) <= 1:
        return pd.DataFrame(lista)
    meio = len(lista) // 2
    esquerda = merge_sort(pd.DataFrame(lista[:meio]), coluna)
    direita = merge_sort(pd.DataFrame(lista[meio:]), coluna)
    return merge(esquerda, direita, coluna)

def merge(esquerda, direita, coluna):
    esquerda_list = esquerda.to_dict('records')
    direita_list = direita.to_dict('records')
    resultado = []
    i = j = 0
    while i < len(esquerda_list) and j < len(direita_list):
        if esquerda_list[i][coluna] > direita_list[j][coluna]:
            resultado.append(esquerda_list[i])
            i += 1
        else:
            resultado.append(direita_list[j])
            j += 1
    resultado.extend(esquerda_list[i:])
    resultado.extend(direita_list[j:])
    return pd.DataFrame(resultado)


# Utilizando problema da mochila
@lru_cache(maxsize=None)
def mochila(i, tempo_restante):
    if i == 0 or tempo_restante == 0:
        return 0
    custo = cursos[i-1]["custo"]
    impacto = cursos[i-1]["impacto"]
    if custo > tempo_restante:
        return mochila(i-1, tempo_restante)
    else:
        incluir = impacto + mochila(i-1, tempo_restante - custo)
        excluir = mochila(i-1, tempo_restante)
        return max(incluir, excluir)


# Recomenda os cursos utilizando o problema da mochila
def recomendar_cursos(tempo_disponivel):
    # Filtra cursos que cabem no tempo disponível
    viaveis = [c for c in cursos if c["custo"] <= tempo_disponivel]

    if not viaveis:
        print("Nenhum curso disponível dentro do tempo informado.")
        return pd.DataFrame([])

    # Calcula a razão impacto/custo para priorizar cursos mais eficientes
    for c in viaveis:
        c["eficiencia"] = round(c["impacto"] / c["custo"])

    # Ordena pelos cursos mais eficientes (impacto maior e custo menor)
    df_viaveis = pd.DataFrame(viaveis)
    df_ordenados = merge_sort(df_viaveis, "eficiencia")

    # Pega os 3 melhores
    top3 = df_ordenados.head(3).reset_index(drop=True)

    impacto_total = top3["impacto"].sum()
    print(f"💡 Impacto total estimado: {impacto_total}")
    print("Cursos recomendados (top 3):")

    return top3

# Relatório
def gerar_relatorio(usuario, cursos_recomendados):
    nome_arquivo = f"relatorio_{usuario['nome']}.txt"
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        f.write(f"Relatório de Cursos Recomendados para {usuario['nome']}\n")
        f.write(f"Nível: {usuario['nivel']} | Tempo disponível: {usuario['tempo']}h/dia\n\n")
        f.write("Cursos sugeridos:\n")
        for _, row in cursos_recomendados.iterrows():
            f.write(f"- {row['nome']} (Impacto: {row['impacto']}, Custo: {row['custo']}h)\n")
    print(f"📄 Relatório gerado: {nome_arquivo}\n")

# ------------------------------
# 6️⃣ Execução principal
# ------------------------------
print("=== Sistema de Recomendação de Cursos ===")
while True:
    opcao = input("1 - Cadastrar | 2 - Login | 0 - Sair: ")
    if opcao == "1":
        cadastrar_usuario()
    elif opcao == "2":
        user = login()
        if user:
            df_ordenado = merge_sort(df_cursos, "impacto")
            print("\n📊 Cursos ordenados por impacto (decrescente):")
            print(df_ordenado)
            recomendados = recomendar_cursos(user["tempo"])
            print(recomendados)
            gerar_relatorio(user, recomendados)
    elif opcao == "0":
        print("Encerrando o sistema...")
        break


=== Sistema de Recomendação de Cursos ===
